# Extracting NBA Draft Data

### realgm.com
**21-08-22**  
Extracting draft data from 2010-2018.  
Keeping this an en extra piece of data. Can't really use in easily in the anlaysis since it't hard to join on Name

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [24]:
#Imports the HTML into python
url = 'https://basketball.realgm.com/nba/draft/past_drafts/2018'
requests.get(url)
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')

In [25]:
#Subsets the HTML to only get the HTML of our table needed
table = soup.find('table', class_ = 'tablesaw')


In [26]:
#Gets all the column headers of our table
# use the initial link
headers = []
for i in table.find_all('th'):
    title = i.text.strip()
    headers.append(title)
headers.append('draft_year')

#### 1st Round

In [27]:
#Creates a dataframe using the column headers from our table
df = pd.DataFrame(columns = headers)
root_link = 'https://basketball.realgm.com/nba/draft/past_drafts/'
years = range(2010,2019)
for year in years:
    #Imports the HTML into python
    url = root_link + str(year)
    requests.get(url)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    
    #Subsets the HTML to only get the HTML of our table needed
    table = soup.find('table', class_ = 'tablesaw')
    
    #gets all our data within the table and adds it to our dataframe
    for j in table.find_all('tr')[1:]:
        row_data = j.find_all('td')
        row = [tr.text for tr in row_data]
        row.append(year)
        length = len(df)
        df.loc[length] = row

In [28]:
df.head()

,Pick,Player,Team,Draft Trades,Pos,HT,WT,Age,YOS,Pre-Draft Team,Class,Nationality,draft_year
0,1,John Wall,WAS,,PG,6-4,210,19,12,Kentucky,Fr *,United States,2010
1,2,Evan Turner,PHL,,GF,6-7,220,21,10,Ohio State,Jr *,United States,2010
2,3,Derrick Favors,NJN,,FC,6-10,265,18,12,Georgia Tech,Fr *,United States,2010
3,4,Wesley Johnson,MIN,,GF,6-7,215,22,9,Syracuse,Jr *,United States,2010
4,5,DeMarcus Cousins,SAC,,C,6-11,270,19,12,Kentucky,Fr *,United States,2010


#### 2nd round

In [29]:
# table = soup.find_all('table', class_ = 'tablesaw')[2]

In [30]:
#Creates a dataframe using the column headers from our table
# df = pd.DataFrame(columns = headers)
years = range(2010,2019)
for year in years:
    #Imports the HTML into python
    url = root_link + str(year)
    requests.get(url)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    
    #Subsets the HTML to only get the HTML of our table needed
    table = soup.find_all('table', class_ = 'tablesaw')[1]
    
    #gets all our data within the table and adds it to our dataframe
    for j in table.find_all('tr')[1:]:
        row_data = j.find_all('td')
        row = [tr.text for tr in row_data]
        row.append(year)
        length = len(df)
        df.loc[length] = row
# clean up Class var
df['Class'] = df.Class.str.replace(' *','', regex = False)
# add round
df.loc[df.Pick.astype(int)<=30,'draft_round'] = 1
df.loc[df.Pick.astype(int)>30,'draft_round'] = 2

In [32]:
df.head()

,Pick,Player,Team,Draft Trades,Pos,HT,WT,Age,YOS,Pre-Draft Team,Class,Nationality,draft_year,draft_round
0,1,John Wall,WAS,,PG,6-4,210,19,12,Kentucky,Fr,United States,2010,1.0
1,2,Evan Turner,PHL,,GF,6-7,220,21,10,Ohio State,Jr,United States,2010,1.0
2,3,Derrick Favors,NJN,,FC,6-10,265,18,12,Georgia Tech,Fr,United States,2010,1.0
3,4,Wesley Johnson,MIN,,GF,6-7,215,22,9,Syracuse,Jr,United States,2010,1.0
4,5,DeMarcus Cousins,SAC,,C,6-11,270,19,12,Kentucky,Fr,United States,2010,1.0


In [34]:
df.to_csv('02_database/nba_draft_realgm.csv',index = False)

### basketball-reference.com
**22-08-22:**  
This export has player_id, which means I can link it to other data. But it doesn't have Class.   
For Class, I will need to work it out based on players' last year in College.  

In [35]:
df = pd.DataFrame(columns = ['pick','team','player_name','college','link','d_year'])

In [36]:
years = range(2003,2020)
for year in years:
    url = 'https://www.basketball-reference.com/draft/NBA_' + str(year) + '.html'
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'lxml')
    table = soup.find('tbody')
    for j in table.find_all('tr'):
        row_data = j.find_all('td')[:4]
        if len(row_data)>0:
            id_ = row_data[2].a.get('href')
            row = [tr.text for tr in row_data]
            row.append(id_)
            row.append(year)
            df.loc[len(df)] = row

# append player_id field
df['nba_player_id'] =  df.link.str.split('/',expand = True)[3].str.split('.',expand = True)[0]

# add round
df.loc[df.pick.astype(int)<=30,'d_round'] = 1
df.loc[df.pick.astype(int)>30,'d_round'] = 2

df.head()    

,pick,team,player_name,college,link,d_year,nba_player_id,d_round
0,1,CLE,LeBron James,,/players/j/jamesle01.html,2003,jamesle01,1.0
1,2,DET,Darko Miličić,,/players/m/milicda01.html,2003,milicda01,1.0
2,3,DEN,Carmelo Anthony,Syracuse,/players/a/anthoca01.html,2003,anthoca01,1.0
3,4,TOR,Chris Bosh,Georgia Tech,/players/b/boshch01.html,2003,boshch01,1.0
4,5,MIA,Dwyane Wade,Marquette,/players/w/wadedw01.html,2003,wadedw01,1.0


In [37]:
df.to_csv('02_database/nba_draft_bbref.csv',index = False)